In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.datasets import mnist

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
m_train = train_images_original.shape[0]
m_test = test_images_original.shape[0]
pixels = train_images_original.shape[1] * train_images_original.shape[2]

In [7]:
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_images.shape, test_images.shape

((60000, 784), (10000, 784))

In [8]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

train_labels = train_labels.reshape(60000, 10)
test_labels = test_labels.reshape(10000, 10)

In [9]:
import torch
import torch.nn.functional as F


In [10]:
def score(X, Y):
    y_hat = model(torch.from_numpy(X))
    predicted = torch.max(y_hat, 1)[1]  
    score = (predicted.numpy() == Y).sum()
    return score/X.shape[0] * 100

In [106]:

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(784, 500)
        self.linear2 = torch.nn.Linear(500,500)
        self.final = torch.nn.Linear(500, 10)
        
        self.softmax = torch.nn.Softmax()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.final(x)
    
        return x



In [107]:
import numpy as np

In [108]:
torch.cuda.is_available()

True

In [121]:
epochs = 500
input_size = pixels
batch_size = 100
output_size = train_labels.shape[1]
lr = .5

dtype = torch.float
device = torch.device("cuda")

In [122]:
model = Model()
model = model.to(device)

criterion = torch.nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.5)

In [123]:
n = train_images.shape[1]
m = train_images.shape[0]
for epoch in range(epochs):
    shuffled_indices = np.random.permutation(m)
    train_images_shuffled = train_images[shuffled_indices]
    train_labels_shuffled = train_labels[shuffled_indices]
    
    for e in range(0,m,batch_size):    
        xi = torch.from_numpy(train_images_shuffled[e:e+batch_size])
        ti = torch.autograd.Variable(torch.from_numpy(train_labels_shuffled[e:e+batch_size]))
        xi = xi.to(device)
        ti = ti.to(device)
        
        optimizer.zero_grad()
        y_pred = model(xi)
        loss = criterion(y_pred, ti)
        loss.backward()

        optimizer.step()
        

    print("Epoch: {} Loss: {}".format(epoch, loss.item()))



Epoch: 0 Loss: 0.035953089594841
Epoch: 1 Loss: 0.023757226765155792
Epoch: 2 Loss: 0.023783154785633087
Epoch: 3 Loss: 0.001546698622405529
Epoch: 4 Loss: 0.002722744829952717
Epoch: 5 Loss: 0.033548515290021896
Epoch: 6 Loss: 0.0005836464697495103
Epoch: 7 Loss: 0.0034548179246485233
Epoch: 8 Loss: 0.0007683114381507039
Epoch: 9 Loss: 0.007282325532287359
Epoch: 10 Loss: 0.002562641864642501
Epoch: 11 Loss: 0.0003270013548899442
Epoch: 12 Loss: 0.0029464627150446177
Epoch: 13 Loss: 7.677829853491858e-05
Epoch: 14 Loss: 0.000853580713737756
Epoch: 15 Loss: 0.0007427419768646359
Epoch: 16 Loss: 0.0004488369741011411
Epoch: 17 Loss: 0.00012046529445797205
Epoch: 18 Loss: 8.525421435479075e-05
Epoch: 19 Loss: 0.0004936876939609647
Epoch: 20 Loss: 0.0001393086859025061
Epoch: 21 Loss: 0.00041491870069876313
Epoch: 22 Loss: 0.000255745166214183
Epoch: 23 Loss: 0.0008873084443621337
Epoch: 24 Loss: 0.00034470934770070016
Epoch: 25 Loss: 0.00033567476202733815
Epoch: 26 Loss: 0.0001760056329

KeyboardInterrupt: 

In [124]:
y_hat = model(torch.from_numpy(test_images).to(device))
predicted = torch.max(y_hat, 1)[1]

In [125]:
score = (predicted.cpu().numpy() == test_labels_original).sum()
score/test_labels_original.shape[0]

0.9842

In [126]:
y_hat = model(torch.from_numpy(train_images).to(device))
predicted = torch.max(y_hat, 1)[1]

In [127]:
score = (predicted.cpu().numpy() == train_labels_original).sum()
score/train_labels_original.shape[0]

1.0

In [128]:
predicted.cpu().numpy()

array([5, 0, 4, ..., 5, 6, 8], dtype=int64)

In [129]:
train_labels_original

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [130]:
y_hat.cpu().detach().numpy()[0]

array([-28.33404  , -25.022905 , -22.584183 ,  -6.2525926, -51.332493 ,
        13.420883 , -44.711666 , -29.602903 , -38.54153  , -36.388783 ],
      dtype=float32)

In [131]:
test_labels.shape

(10000, 10)